In [5]:
from model import build_gpt_model
from sklearn.model_selection import train_test_split
import torch
from dataset import GPTChatDataset
from tokenizer_utils import TokenizerEnVi
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN = 256

In [7]:


from convert import load
lines = load()


In [8]:
print(len(lines))

145206


In [9]:
train_lines, test_lines = train_test_split(lines, test_size=0.2, random_state = 42)


In [10]:
print(len(train_lines))

116164


In [11]:
tokenizer = TokenizerEnVi()
tokenizer.build_vocab(train_lines)

pad_id = tokenizer.word2idx["<pad>"]

train_dataset = GPTChatDataset(train_lines, tokenizer, seq_len= SEQ_LEN )
test_dataset = GPTChatDataset(test_lines, tokenizer, seq_len= SEQ_LEN)



In [12]:
from statistics import mean

lengths = [len(tokenizer.tokenize(line)) for line in lines]
print("Trung bình:", mean(lengths), "| Max:", max(lengths), "| Min:", min(lengths))

Trung bình: 128.83127418977176 | Max: 1552 | Min: 4


In [13]:
import pickle

with open("saved/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [14]:
def collate_fn(batch):
    decoder_input = torch.stack([item["decoder_input"] for item in batch])
    decoder_mask = torch.stack([item["decoder_mask"] for item in batch])
    label = torch.stack([item["label"] for item in batch])

    return {
        "decoder_input": decoder_input,      # (batch_size, seq_len)
        "decoder_mask": decoder_mask,        # (batch_size, 1, seq_len, seq_len)
        "label": label                       # (batch_size, seq_len)
    }

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [15]:
for word in tokenizer.word2idx:
    print(word)

<pad>
<unk>
<sos>
<eos>
USER
chi_phí
sống
ở
vancouver
là
bao_nhiêu
?
AI
sinh_hoạt
thay_đổi
dựa
trên
một_số
yếu_tố
bao_gồm
nhà
,
giao_thông
thực_phẩm
và
các
khác
theo
numbeo
cung_cấp
chỉ_số
sinh_hoạt_tính
đến
tháng
năm
hàng
ước_tính
cho
một
người
mà
không
cần
thuê
khoảng
cad
căn_hộ
phòng
ngủ
trung_tâm
thành_phố
mỗi
.
điều
quan_trọng
lưu_ý
đây
trung_bình
thực_tế
có_thể
lối
hoàn_cảnh
cá_nhân
xác_định
câu
trả_lời
đúng
từ
lựa_chọn
bên
dưới
ví_dụ
về
độc_quyền
a
google
b
apple
c
microsoft
d
twitter
có
trong
hoặc
thuần
túy
hãy
đưa
ra
nguồn
dữ_liệu
được
sử_dụng
mô_hình_học
máy
bộ
sưu_tập
hình_ảnh
để
huấn_luyện
nhằm
nhận_diện
đối_tượng
bức
ảnh
thực_hiện
khuôn_mặt
phát_hiện
chuyển_động
văn_bản
tài_liệu
video
âm_thanh
cấu_trúc
như
giá
cổ_phiếu
thời_tiết
tôi
ngạn_ngữ
ngôn_ngữ
sau
tiếng
pháp
nổi_tiếng
l
habit
ne
fait
pas
le
moine
nó
thường
dịch
sang
anh
quần_áo
làm
đàn_ông
có_nghĩa_là
bạn
không_thể
đánh_giá
tính_cách
của
ai
đó
ngoại_hình
họ
tạo
lời
kêu_gọi
hành_động
bài
viết
này
nhấn_mạnh
sự
cần_thi

In [16]:
model = build_gpt_model(len(tokenizer.word2idx), seq_len= SEQ_LEN).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [17]:
n_epochs = 10
best_val_loss = float("inf")
save_path     = "saved/best_model.pth"
os.makedirs("saved", exist_ok=True)
for epoch in range(n_epochs):
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_loader, desc=f"[Epoch {epoch}] Validating"):
        decoder_input = batch["decoder_input"].to(device)
        decoder_mask = batch["decoder_mask"].to(device)
        labels = batch["label"].to(device)

        # Forward
        output = model(decoder_input, decoder_mask)  

        # Loss
        output = output.view(-1, output.shape[-1])
        labels = labels.view(-1)
        loss = criterion(output, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    
    
    # Evaluation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            decoder_input = batch["decoder_input"].to(device)
            decoder_mask = batch["decoder_mask"].to(device)
            labels = batch["label"].to(device)


            output = model(decoder_input, decoder_mask)  

            output = output.view(-1, output.shape[-1])
            labels = labels.view(-1)
            loss = criterion(output, labels)

            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)
    
    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    
    # Save model if best
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)
        print("Saved best model.")


[Epoch 0] Validating:  15%|█▍        | 529/3631 [52:30<5:07:56,  5.96s/it]


KeyboardInterrupt: 